<a href="https://colab.research.google.com/github/gkiflex/MSAI-630-A01/blob/master/long_short_term_memory_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check TensorFlow version (should be 2.x)
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

# Import required libraries
import numpy as np
import json
import re
import string
from tensorflow.keras import layers, models, callbacks, losses

TensorFlow version: 2.19.0


In [2]:
# Install kaggle library
!pip install kaggle

# Upload your kaggle.json file (from Kaggle → Account → API → Create New API Token)
from google.colab import files
uploaded = files.upload()  # Upload your kaggle.json file

# Set up Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d hugodarwood/epirecipes
!unzip epirecipes.zip

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/hugodarwood/epirecipes
License(s): unknown
  0% 0.00/11.3M [00:00<?, ?B/s]
100% 11.3M/11.3M [00:00<00:00, 1.25GB/s]
Archive:  epirecipes.zip
  inflating: epi_r.csv               
  inflating: full_format_recipes.json  
  inflating: recipe.py               
  inflating: utils.py                


In [4]:
VOCAB_SIZE = 10000
MAX_LEN = 200
EMBEDDING_DIM = 100
N_UNITS = 128
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 25  # You might want to reduce this for testing (e.g., 5-10)


In [5]:
# Load the dataset - adjust path based on your file location
with open("full_format_recipes.json") as json_data:  # Update this path
    recipe_data = json.load(json_data)

# Filter the dataset
filtered_data = [
    "Recipe for " + x["title"] + " | " + " ".join(x["directions"])
    for x in recipe_data
    if "title" in x
    and x["title"] is not None
    and "directions" in x
    and x["directions"] is not None
]

# Count the recipes
n_recipes = len(filtered_data)
print(f"{n_recipes} recipes loaded")

# Display example
example = filtered_data[9]
print("Example recipe:")
print(example[:200] + "..." if len(example) > 200 else example)

20111 recipes loaded
Example recipe:
Recipe for Ham Persillade with Mustard Potato Salad and Mashed Peas  | Chop enough parsley leaves to measure 1 tablespoon; reserve. Chop remaining leaves and stems and simmer with broth and garlic in ...


In [6]:
# Pad the punctuation
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}])", r" \\1 ", s)
    s = re.sub(" +", " ", s)
    return s

text_data = [pad_punctuation(x) for x in filtered_data]

# Convert to TensorFlow Dataset
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

# Create vectorization layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

# Adapt the layer
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

# Display some token mappings
print("Token mappings:")
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

Token mappings:
0: 
1: [UNK]
2: \1
3: and
4: to
5: in
6: the
7: with
8: a
9: until


In [7]:
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y

train_ds = text_ds.map(prepare_inputs)

In [8]:
inputs = layers.Input(shape=(None,), dtype="int32")
x = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x = layers.LSTM(N_UNITS, return_sequences=True)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
lstm = models.Model(inputs, outputs)
lstm.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 100)      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 128)      │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 10000)    │     1,290,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,407,248 (9.18 MB)

 Trainable params: 2,407,248 (9.18 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append({"prompt": start_prompt, "word_probs": probs})
            start_tokens.append(sample_token)
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\\nGenerated text:\\n{start_prompt}\\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 5 == 0:  # Generate text every 5 epochs
            self.generate("recipe for", max_tokens=50, temperature=1.0)

text_generator = TextGenerator(vocab)

In [11]:
# Compile the model
loss_fn = losses.SparseCategoricalCrossentropy()
lstm.compile("adam", loss_fn)

# Set up callbacks - FIXED: Use .weights.h5 extension
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint.weights.h5",
    save_weights_only=True,
    save_freq="epoch",
    verbose=1,
)

# Train the model
history = lstm.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, text_generator],
)

Epoch 1/25
629/629 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 4.8190
Epoch 1: saving model to ./checkpoint.weights.h5
\nGenerated text:\nrecipe for inchdiameter easter chipotle salad \1 until a mill add onion \1 highball chicken ingredients \1 15 occasionally \1 at teaspoons parchment \1 add sugar \1 add knead drizzled along \1 onion with cook bowls \1 in paper thickens \1 whisk stir heat until smooth \1 skin beans \1\n
629/629 ━━━━━━━━━━━━━━━━━━━━ 32s 46ms/step - loss: 4.8176
Epoch 2/25
628/629 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 3.0454
Epoch 2: saving model to ./checkpoint.weights.h5
629/629 ━━━━━━━━━━━━━━━━━━━━ 26s 41ms/step - loss: 3.0449
Epoch 3/25
628/629 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 2.5293
Epoch 3: saving model to ./checkpoint.weights.h5
629/629 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - loss: 2.5291
Epoch 4/25
629/629 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 2.3114
Epoch 4: saving model to ./checkpoint.weights.h5
629/629 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - loss: 2

In [12]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        print(f"\\nPROMPT: {i['prompt']}")
        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \\t{np.round(100*p,2)}%")
        print("--------\\n")

# Test with temperature = 0.5 (more conservative)
print("=== TEMPERATURE 0.5 (Conservative) ===")
info_05 = text_generator.generate(
    "recipe for chocolate cake |", max_tokens=30, temperature=0.5
)

# Test with temperature = 1.5 (more creative)
print("\\n=== TEMPERATURE 1.5 (Creative) ===")
info_15 = text_generator.generate(
    "recipe for chocolate cake |", max_tokens=30, temperature=1.5
)

# Test another prompt
print("\\n=== TEMPERATURE 0.7 (Balanced) ===")
info_07 = text_generator.generate(
    "recipe for pasta with", max_tokens=25, temperature=0.7
)

=== TEMPERATURE 0.5 (Conservative) ===
\nGenerated text:\nrecipe for chocolate cake | \1 preheat oven to 350°f \1 butter 13x9x2 \1 inch glass baking dish \1 stir 1 \1 4 cup sugar and 1 \1 2 cup\n
\n=== TEMPERATURE 1.5 (Creative) ===
\nGenerated text:\nrecipe for chocolate cake | \1 whisk honey to egg in an electric mixer filled pancake with eggs in a prepared madeleine or enamel glass \1 lemons \1 then avoiding\n
\n=== TEMPERATURE 0.7 (Balanced) ===
\nGenerated text:\nrecipe for pasta with wild sauce \1 melt butter in heavy large skillet over medium heat \1 add onion \1 sauté until onions are tender\n
